In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import langchain
import faiss
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-10-13 13:18:12.576647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760361492.603015     101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760361492.610757     101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!wget -q --show-progress -O all-the-news-3.zip "https://www.dropbox.com/scl/fi/wruzj2bwyg743d0jzd7ku/all-the-news-3.zip?rlkey=rgwtwpeznbdadpv3f01sznwxa&dl=1"

!unzip all-the-news-3.zip

all-the-news-3.zip  100%[===================>] 166.04M  22.4MB/s    in 7.3s    
Archive:  all-the-news-3.zip
  inflating: all-the-news-3.csv      


In [11]:
df = pd.read_csv('./all-the-news-3.csv', nrows=1000)
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [25]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Build embedding on news titles

In [5]:
emb_dim = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatIP(emb_dim)

In [6]:
titles = df["title"].tolist()

In [7]:
print(len(titles))

100000


In [8]:
emb = model.encode(titles, normalize_embeddings=True, convert_to_numpy=True)
index.add(emb)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

In [9]:
def get_recommendations(search_term, k=5):
    svec = model.encode(search_term, normalize_embeddings=True, convert_to_numpy=True)
    scores, ids = index.search(svec.reshape(1, -1), k)
    for s, idx in zip(scores[0], ids[0]):
        print(f"{s:.3f}  {titles[idx]}")

In [10]:
get_recommendations("obama", k=10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.680  Barack Obama, the President That Punk Gave a Pass To
0.639  Obama: Trump's a Good Choice for Republicans!!!
0.594  Obama: I didn’t appreciate how weak the presidency is until I was president
0.592  Barack Obama has a plan for the Trump era
0.590  The real target of Trump’s inaugural speech wasn’t Barack Obama. It was George W. Bush.
0.585  The core contradiction of Barack Obama’s post-presidency
0.575  Don’t call this man the “next Obama”
0.575  The most powerful moment from Obama’s last State of the Union
0.568  Watch: Barack Obama’s final press conference as president
0.561  President Barack Obama: His Own Blood's Voting for Donald Trump


# Create embedding on all news content

In [26]:
emb_dim = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatIP(emb_dim)

In [27]:
articles = df["article"].tolist()
titles = df["title"].tolist()

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)

all_chunks, all_titles = [], []          # same length, same order
for title, art in zip(titles, articles):
    if isinstance(art, str):
        chunks = text_splitter.split_text(art)
        all_chunks.extend(chunks)
        all_titles.extend([title]*len(chunks))   # duplicate title per chunk

# embed once
embs = model.encode(all_chunks, normalize_embeddings=True, convert_to_numpy=True)
index.add(embs.astype("float32"))

Batches:   0%|          | 0/335 [00:00<?, ?it/s]

In [29]:
def retrieve(query, k=5):
    qvec = model.encode([query], normalize_embeddings=True)
    scores, ids = index.search(qvec.astype("float32"), k)
    results = []
    for score, idx in zip(scores[0], ids[0]):
        results.append({
            "title": all_titles[idx],
            "chunk": all_chunks[idx],
            "score": float(score)
        })
    return results

In [30]:
for hit in retrieve("obama", 10):
    print(f"{hit['score']:.3f}  |  {hit['title']}  |  {hit['chunk'][:100]}…")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.540  |  Obama-Trump meeting 'less awkward' than some might have expected: White House  |  WASHINGTON (Reuters) - U.S. President Barack Obama’s Oval Office meeting on Thursday with his electe…
0.527  |  In separate speeches Obama, Biden, Carter target Trump policies  |  WASHINGTON (Reuters) - President Barack Obama, his vice president and defense secretary took aim on …
0.495  |  Government shutdown: are Democrats becoming more like Republicans?  |  On October 7, 2013, President Barack Obama took the podium at the headquarters of the Federal Emerge…
0.495  |  It's 2018, and Trump is still tweeting about Hillary Clinton's email  |  Donald Trump’s highly abnormal presidency: the week of Jan. 1  Donald Trump’s highly abnormal presid…
0.493  |  In separate speeches Obama, Biden, Carter target Trump policies  |  of Muslims and to profile Muslim Americans reflected the politics of fear and intolerance.  “There a…
0.490  |  Medicare-for-all will be shaped by these 3 questions  |  to Obama, p

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)

print(len(articles),len(titles))

texts, chunked_titles = [], []

for title, article in zip(titles, articles): 
    if article is not None and isinstance(article, str):
        chunks = text_splitter.split_text(article)
        chunks_emb = model.encode(chunks, normalize_embeddings=True, convert_to_numpy=True)
        index.add(chunks_emb)
        texts.extend(chunks)
        chunked_titles.extend([title]*len(chunks))

100000 100000


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def retrieve(query, k=5):
    qvec = model.encode([query], normalize_embeddings=True)
    scores, ids = index.search(qvec.astype("float32"), k)
    return [(chunked_titles[i], texts[i], float(scores[0][j])) for j, i in enumerate(ids[0])]

In [ ]:
retrieve("obama", k=10)